In [1]:
from sklearn.datasets import fetch_covtype
from sklearn.model_selection import cross_val_score, StratifiedKFold

In [3]:
covertype_dataset = fetch_covtype(random_state=101, shuffle=True)

In [4]:
covertype_dataset.target = covertype_dataset.target.astype(int)
covertype_X = covertype_dataset.data[:15000,:]
covertype_y = covertype_dataset.target[:15000] -1 
covertype_val_X = covertype_dataset.data[15000:20000,:]
covertype_val_y = covertype_dataset.target[15000:20000] -1
covertype_test_X = covertype_dataset.data[20000:25000,:]
covertype_test_y = covertype_dataset.target[20000:25000] -1

## XGBoost

In [7]:
import xgboost as xgb

hypothesis = xgb.XGBClassifier(objective= "multi:softprob", 
                               max_depth=24, 
                               gamma=0.1, 
                               subsample=0.90,
                               learning_rate=0.01, 
                               n_estimators = 500, 
                               nthread=-1)

hypothesis.fit(covertype_X, covertype_y, 
               eval_set=[(covertype_val_X, covertype_val_y)], 
               eval_metric='merror', 
               early_stopping_rounds=25, 
               verbose=False) 


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0.1, learning_rate=0.01, max_delta_step=0,
       max_depth=24, min_child_weight=1, missing=None, n_estimators=500,
       n_jobs=1, nthread=-1, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.9)

In [8]:
from sklearn.metrics import accuracy_score, confusion_matrix
preds = hypothesis.predict(covertype_test_X)
print('test accuracy:', accuracy_score(covertype_test_y, preds))
print(confusion_matrix(covertype_test_y, preds))

test accuracy: 0.848
[[1512  288    0    0    0    2   18]
 [ 215 2197   18    0    7   11    0]
 [   0   17  261    4    0   19    0]
 [   0    0    4   20    0    3    0]
 [   1   54    3    0   19    0    0]
 [   0   16   42    0    0   86    0]
 [  37    1    0    0    0    0  145]]


C:\Users\Luca\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


## LightGBM

In [50]:
import lightgbm as lgb
import numpy as np

In [60]:
params = {'task': 'train',
          'boosting_type': 'gbdt',
          'objective': 'multiclass',
          'num_class':len(np.unique(covertype_y)),
          'metric': 'multi_logloss',
          'learning_rate': 0.01,
          'max_depth': 128,
          'num_leaves': 256,
          'feature_fraction': 0.9,
          'bagging_fraction': 0.9,
          'bagging_freq': 10}

In [61]:
train_data = lgb.Dataset(data=covertype_X, label=covertype_y)
val_data = lgb.Dataset(data=covertype_val_X, label=covertype_val_y)

In [62]:
bst = lgb.train(params, 
                 train_data, 
                 num_boost_round=2500, 
                 valid_sets=val_data,
                 verbose_eval=500,
                 early_stopping_rounds=25)

Training until validation scores don't improve for 25 rounds.
[500]	valid_0's multi_logloss: 0.435976
Early stopping, best iteration is:
[851]	valid_0's multi_logloss: 0.400478


In [65]:
lgb_cv = lgb.cv(params, 
                 train_data, 
                 num_boost_round=2500, 
                 nfold=3, 
                 shuffle=True, 
                 stratified=True, 
                 verbose_eval=500, 
                 early_stopping_rounds=25)

nround = lgb_cv['multi_logloss-mean'].index(np.min(lgb_cv['multi_logloss-mean']))
print("Best number of rounds: %i" % nround)

[500]	cv_agg's multi_logloss: 0.468806 + 0.0124661
Best number of rounds: 782


In [63]:
y_probs = bst.predict(covertype_test_X, num_iteration=bst.best_iteration)
y_preds = np.argmax(y_probs, axis=1)

In [64]:
from sklearn.metrics import accuracy_score, confusion_matrix
print('test accuracy:', accuracy_score(covertype_test_y, y_preds))
print(confusion_matrix(covertype_test_y, y_preds))

test accuracy: 0.8444
[[1495  309    0    0    0    2   14]
 [ 221 2196   17    0    5    9    0]
 [   0   20  258    5    0   18    0]
 [   0    0    3   19    0    5    0]
 [   1   51    4    0   21    0    0]
 [   0   14   43    0    0   87    0]
 [  36    1    0    0    0    0  146]]


## CatBoost

In [1]:
import numpy as np
from sklearn.datasets import fetch_covtype
from catboost import CatBoostClassifier, Pool

In [ ]:
# https://tech.yandex.com/catboost/doc/dg/concepts/python-usages-examples-docpage/#load-from-list-ndarray-dataframe-series
# https://tech.yandex.com/catboost/doc/dg/concepts/python-usages-examples-docpage/#multiclassification
# https://tech.yandex.com/catboost/doc/dg/concepts/python-reference_parameters-list-docpage/

In [2]:
covertype_dataset = fetch_covtype(random_state=101, shuffle=True)

In [3]:
label = covertype_dataset.target.astype(int) - 1

wilderness_area = np.argmax(covertype_dataset.data[:,10:(10+4)], axis=1)
soil_type = np.argmax(covertype_dataset.data[:,(10+4):(10+4+40)], axis=1)
data = (covertype_dataset.data[:,:10], wilderness_area.reshape(-1,1), soil_type.reshape(-1,1))
data = np.hstack(data)

In [4]:
covertype_train = Pool(data[:15000,:], label[:15000], [10, 11])
covertype_val = Pool(data[15000:20000,:], label[15000:20000], [10, 11])
covertype_test = Pool(data[20000:25000,:], None, [10, 11])
covertype_test_y = label[20000:25000]

In [5]:
model = CatBoostClassifier(iterations=4000, 
                           learning_rate=0.05, 
                           depth=8,
                           custom_loss = 'Accuracy',
                           eval_metric = 'Accuracy',
                           use_best_model=True,
                           loss_function='MultiClass')

In [6]:
model.fit(covertype_train, eval_set=covertype_val, verbose=False, plot=True)

MetricVisualizer(data={'catboost_info': {'path': 'catboost_info', 'name': 'catboost_info', 'content': {'passed…

In [7]:
preds_class = model.predict(covertype_test)
preds_proba = model.predict_proba(covertype_test)

In [8]:
from sklearn.metrics import accuracy_score, confusion_matrix
print('test accuracy:', accuracy_score(covertype_test_y, preds_class))
print(confusion_matrix(covertype_test_y, preds_class))

test accuracy: 0.847
[[1482  320    0    0    0    0   18]
 [ 213 2199   12    0   10   12    2]
 [   0   13  260    5    0   23    0]
 [   0    0    6   18    0    3    0]
 [   2   40    5    0   30    0    0]
 [   0   16   33    1    0   94    0]
 [  31    0    0    0    0    0  152]]
